# Scrape Vb Google Scholar

In [46]:
import __main__ as main
import subprocess as sp
import time
from bs4 import BeautifulSoup
from tqdm import tqdm
import re
notebook_name = 'scrape_vb_google_scholar'

if not hasattr(main, '__file__'):
    sp.run(f"jupyter nbconvert --to script '{notebook_name}.ipynb'; chmod u+x {notebook_name}.py", shell=True)

[NbConvertApp] WARNING | Config option `extra_template_paths` not recognized by `ScriptExporter`.  Did you mean `template_path`?
[NbConvertApp] Converting notebook scrape_vb_google_scholar.ipynb to script
[NbConvertApp] WARNING | Config option `extra_template_paths` not recognized by `PythonExporter`.  Did you mean `template_path`?
[NbConvertApp] Writing 2878 bytes to scrape_vb_google_scholar.py


In [8]:
vineet_google_scholar = 'https://scholar.google.com/citations?hl=en&user=zr2I_WMAAAAJ&view_op=list_works&sortby=pubdate'
sp.run(f"cd publications; wget '{vineet_google_scholar}' -O gscholar_hits.html", shell=True)

--2024-10-23 23:00:42--  https://scholar.google.com/citations?hl=en&user=zr2I_WMAAAAJ&view_op=list_works&sortby=pubdate
Resolving scholar.google.com (scholar.google.com)... 142.250.72.164
Connecting to scholar.google.com (scholar.google.com)|142.250.72.164|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: 'gscholar_hits.html'

     0K .......... .......... .......... .......... .......... 1.06M
    50K .......... .......... .......... .......... .......... 1.75M
   100K .......... .......... .......... .......... .......    3.52M=0.09s

2024-10-23 23:00:43 (1.65 MB/s) - 'gscholar_hits.html' saved [150715]



CompletedProcess(args="cd publications; wget 'https://scholar.google.com/citations?hl=en&user=zr2I_WMAAAAJ&view_op=list_works&sortby=pubdate' -O gscholar_hits.html", returncode=0)

In [9]:
with open("publications/gscholar_hits.html", 'rb') as infile:
    hits = infile.read()
hits = hits.decode('unicode-escape')

/var/folders/hg/fb033z3906sbs5qrlt7sc2sm0000gn/T/ipykernel_79530/177708489.py:3: DeprecationWarning: invalid escape sequence '\S'
  hits = hits.decode('unicode-escape')


In [19]:
publications = re.findall("(?=<td class=\"gsc_a_t\">).*?(?=</span>)", hits)
authors_ = [re.search('(?<=<div class=\"gs_gray\">).*?(?=</div>)', x).group(0) for x in publications]
titles = [re.search('(?<=class=\"gsc_a_at\">).*?(?=</a>)', x).group(0) for x in publications]
years = [re.search('20[0-9][0-9]', x).group(0) for x in publications]
citation_links = ['https://scholar.google.com/citations?view_op=view_citation&' + re.search('(?=citation_for_view).*?(?=" )', x).group(0) for x in publications]

In [42]:
authors = []
dates = []
journals = []
links = []
for citation_link in tqdm(citation_links):
    sp.run(f"wget '{citation_link}' -O tmp.html", shell=True)
    with open("tmp.html", 'rb') as infile:
        lines = infile.read().decode('unicode-escape')
        author_list, date, journal, *_ = re.findall('(?<=gsc_oci_value">).*?(?=</div>)', lines)
        if '"' in journal:
            journal = ''
        link = re.search('(?<=class="gsc_oci_title_link" href=").*?(?=")', lines).group(0)
        authors.append(BeautifulSoup(author_list).text)
        dates.append(BeautifulSoup(date).text)
        journals.append(journal)
        links.append(link)
        
    time.sleep(5)
    sp.run("rm tmp.html", shell=True)

  0%|                                    | 0/20 [00:00<?, ?it/s]--2024-10-23 23:10:07--  https://scholar.google.com/citations?view_op=view_citation&citation_for_view=zr2I_WMAAAAJ:bKqednn6t2AC
Resolving scholar.google.com (scholar.google.com)... 142.250.72.132
Connecting to scholar.google.com (scholar.google.com)|142.250.72.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: 'tmp.html'

     0K .......... .......... .......... .......... .......... 1.11M
    50K .......... .......... .......... ..                    1.29M=0.07s

2024-10-23 23:10:08 (1.17 MB/s) - 'tmp.html' saved [84848]

/var/folders/hg/fb033z3906sbs5qrlt7sc2sm0000gn/T/ipykernel_79530/4049400432.py:8: DeprecationWarning: invalid escape sequence '\S'
  lines = infile.read().decode('unicode-escape')
/var/folders/hg/fb033z3906sbs5qrlt7sc2sm0000gn/T/ipykernel_79530/4049400432.py:14: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. Y

 60%|████████████████▏          | 12/20 [01:07<00:44,  5.54s/it]--2024-10-23 23:11:15--  https://scholar.google.com/citations?view_op=view_citation&citation_for_view=zr2I_WMAAAAJ:zA6iFVUQeVQC
Resolving scholar.google.com (scholar.google.com)... 142.250.72.132
Connecting to scholar.google.com (scholar.google.com)|142.250.72.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: 'tmp.html'

     0K .......... .......... .......... .......... ..........  975K
    50K .......... .......... .......... ...                   2.23M=0.07s

2024-10-23 23:11:15 (1.24 MB/s) - 'tmp.html' saved [85360]

 65%|█████████████████▌         | 13/20 [01:12<00:38,  5.56s/it]--2024-10-23 23:11:20--  https://scholar.google.com/citations?view_op=view_citation&citation_for_view=zr2I_WMAAAAJ:MLfJN-KU85MC
Resolving scholar.google.com (scholar.google.com)... 142.250.72.132
Connecting to scholar.google.com (scholar.google.com)|142.250.72.132|:443... connecte

In [43]:
title_length_limit = 75
abbreviated_titles = [title if len(title) < title_length_limit else title[:title_length_limit-3]+'...' for title in titles]

In [44]:
paper_list = []
for title, author_list, link, journal, year in zip(abbreviated_titles, authors_, links, journals, years):
    paper_list.append('\t\t\t' + f"""<li><div class=link>
\t\t\t\t<a href=\"{link}\">{title}</a>
\t\t\t\t<br>
\t\t\t\t<span>{author_list}</span>
\t\t\t\t<br>
\t\t\t\t<span>{journal}, {year}</span>
\t\t\t\t</li>
""".strip())
paper_list = '\n'.join(paper_list)

In [45]:
with open('index.html') as infile:
    lines = infile.read()
    start_papers = lines.index('<ul>') + len('<ul>\n')
    end_papers = lines.index('</ul>') - len('\n\t\t')

with open('index.html', 'w') as outfile:
    outfile.write(lines[:start_papers] + paper_list + lines[end_papers:])